In [1]:
from google.cloud import storage
import pickle
import requests
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build # google-api-python-client
import os

In [2]:
# TODO: Will need to update this for Gcloud
# New token
# Don't need storage creds
TOKEN = open('../token.txt', 'rb').read().decode('utf-8')
try:
    CLIENT = storage.Client()
except:
    CLIENT = storage.Client.from_service_account_json('../royaleapp-296a6cea39ad.json')
BUCKET = CLIENT.bucket('royale-data')

In [19]:
def read_player_tags():
    blob = BUCKET.get_blob('credentials/2020_04_23_gsheet_credentials.json')
    blob.download_to_filename('creds.json')
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = Credentials.from_service_account_file('creds.json', scopes=scope)
    os.remove('creds.json')
    service = build('sheets', 'v4', credentials=credentials)
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId='1cmSIq5-6NI5Bn1n8WTfaVTYthxTIRUQ8aNsDTixsHno',
                                range='Sheet1').execute()
    return result

def process_battles(player_tag):
    # read in data from storage
    # if no data, create data
    blob = BUCKET.get_blob('user_data/{}.p'.format(player_tag))
    if blob is None:
        data = {
            'battle_time':[],
            'team_cards':[],
            'opponent_cards':[],
            'team_trophy_count':[],
            'game_mode':[],
            'arena':[],
            'win_loss':[]
        }
    else:
        data = pickle.loads(blob.download_as_string())
    # process battles and add to data
    response = requests.get('https://api.clashroyale.com/v1/players/%23{}/battlelog'.format(player_tag),
                            headers={"Authorization": "Bearer {}".format(TOKEN)})
    battle_list = eval(response.text.replace('false', 'False').replace('true', 'True'))
    for battle in battle_list:
        # make sure you don't add the same battles twice
        # battles in 2v2, where it's harder to get data for both teams, are excluded
        if (battle['battleTime'] not in data['battle_time']) and ('2v2' not in battle['type']):
            data['battle_time'].append(battle['battleTime'])
            data['team_cards'].append([card['name'] for card in battle['team'][0]['cards']])
            data['opponent_cards'].append([card['name'] for card in battle['opponent'][0]['cards']])
            data['team_trophy_count'].append(battle['team'][0]['startingTrophies'])
            data['game_mode'].append("{} - {}".format(battle['type'], battle['gameMode']['name']))
            data['arena'].append(battle['arena']['name'])
            if battle['team'][0]['crowns'] > battle['opponent'][0]['crowns']:
                data['win_loss'].append('win')
            elif battle['team'][0]['crowns'] < battle['opponent'][0]['crowns']:
                data['win_loss'].append('loss')
            else:
                assert 1 == 2
#     # save to storage
#     local_fname = '{}.p'.format(player_tag)
#     with open(local_fname, 'wb') as f:
#         pickle.dump(data, f)
#     BUCKET.blob('user_data/{}.p'.format(player_tag)).upload_from_file(open(local_fname, 'rb'))
#     os.remove(local_fname)
    
def main():
    result = read_player_tags()
    for val in result['values']:
        player_tag = val[0]
        process_battles(player_tag)
    

In [23]:
# Check how many battles I have right now
blob = BUCKET.get_blob('user_data/{}.p'.format('9Q9GY29CQ'))
data = pickle.loads(blob.download_as_string())
len(data['win_loss'])

53

In [20]:
result = read_player_tags()

In [22]:
%%timeit
process_battles(result['values'][0][0])

554 ms ± 93.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Can do about 1000 users in 9 minutes: Start with this through testing

# For full launch: How much does this cost?
# Will need to put this into AI Jobs and trigger the Job through the cloud function
# Can I paraellelize in AI Jobs?

In [2]:
# Do it from Storage instead

In [3]:
client = storage.Client()
bucket = client.bucket('royale-data')

In [47]:
blobs = bucket.list_blobs(prefix='user_data')

In [48]:
[blob.name.split('/')[1][:-2] for blob in blobs if blob.name.endswith('.p')]

['9Q9GY29CQ', 'P9VUGYL8L']